<a href="https://colab.research.google.com/github/statisticianinstilettos/Hackathon-Assets/blob/main/wildlife_video_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/static/img/opik-logo.svg" width="300"/>






In [2]:
!pip install opik

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.8 MB/s eta 0:00:00


In [3]:
!opik configure

Which Opik deployment do you want to log your traces to?
1 - Opik Cloud (default)
2 - Self-hosted Comet platform
3 - Local deployment

> 1
OPIK: Your Opik API key is available in your account settings, can be found at https://www.comet.com/api/my/settings/ for Opik cloud
Please enter your Opik API key:
Do you want to use "statisticianinstilettos" workspace? (Y/n)Y
OPIK: Configuration saved to file: /root/.opik.config
OPIK: Configuration completed successfully. Traces will be logged to 'Default Project' project. To change the destination project, see: https://www.comet.com/docs/opik/tracing/log_traces#configuring-the-project-name


In [4]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [16]:
import opik
from opik import track
from opik.integrations.openai import track_openai
from openai import OpenAI

os.environ["OPIK_PROJECT_NAME"] = "wildlife_video_project"

client = OpenAI()
tracked_client = track_openai(client)

In [26]:
#Research Agent

@track
def research_agent(animal: str, location: str, n_facts: int = 3) -> list[str]:
    """
    Generate short, engaging animal facts suitable for a wildlife video.
    """

    research_agent_prompt = f"""
    Generate {n_facts} short, fascinating facts about {animal} in {location}.
    Requirements:
    - Each fact must be 1 sentence
    - Written for a general audience
    - No emojis
    - No anthropomorphism
    """

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "You are a wildlife education expert."},
            {"role": "user", "content": research_agent_prompt},
        ],
        temperature=0.7,
    )

    facts_text = response.choices[0].message.content
    facts = [f.strip("- ").strip() for f in facts_text.split("\n") if f.strip()]

    return facts

In [27]:
#Video Agent

@track
def video_agent(prompt: str) -> dict:
    """Generate a video using OpenAI's Sora model."""
    # Create video and wait for completion
    video = tracked_client.videos.create_and_poll(
        model="sora-2",
        prompt=prompt,
    )

    result = {"id": video.id, "status": video.status}

    # Download the video if generation succeeded
    # If OpenAI moderation rejected your request - you'll see it in the create_and_poll span output
    if video.status == "completed":
        content = tracked_client.videos.download_content(video_id=video.id)
        content.write_to_file("output_video.mp4")
        result["output_path"] = "output_video.mp4"

    return result


In [28]:
#Orchestration

@track
def agent_orchestrator(location: str, animal: str):

    facts = research_agent(animal, location)

    video_prompt = f"""
    Create a cinematic wildlife video of a {animal} in its natural habitat {location}.
    Narrate the video with the following facts:
    {facts}
    """

    video_result = video_agent(video_prompt)

    return {
        "animal": animal,
        "location": location,
        "facts": facts,
        "video": video_result
    }

In [29]:
# Generate final output
agent_orchestrator(location="Las Vegas, Nevada", animal="Roadrunner")

{'animal': 'Roadrunner',
 'location': 'Las Vegas, Nevada',
 'facts': ['1. The Greater Roadrunner can sprint up to 20 miles per hour, helping it catch prey like insects and small reptiles in the desert around Las Vegas.',
  '2. Unlike many birds, the roadrunner can survive without drinking water for long periods, obtaining moisture from the food it eats.',
  '3. Roadrunners use their strong legs to stomp prey before swallowing it whole, a unique hunting method among desert birds.'],
 'video': {'id': 'video_697d264b0f8c8191864bdcace55653ef087e4e096ed77dbd',
  'status': 'completed',
  'output_path': 'output_video.mp4'}}